# Recurrent Neural Networks

All the models we have discussed so far were looking at the inputs as isolated instances. In image classification we determine the category of and image, in semantic segmentation we produce a semantic map from the input. But there are many cases were datapoints are not isolated instances and have connection to each other. Sequential data are the type of data where each instance is related to the instances came before. A good example for this type of data is time series data. At each point in time to the value of the time series depends on the value of the prior points. Recurrent Neural Networks (RNN) are a class of networks which deal with sequential data. There are many variants of Recurrent Neural Networks, including:

- Simple Recurrect Neural Networks (Simple RNN - or often just called RNN)
- Long Short-Term Memory (LSTM)
- Gated Recurrent Unit (GRU)

In this notebook we will discuss LSTM; however, the general logic behind all these methods are the same. They only differ in the way they handel information internally. 
<img src='./images/RNN.png'>
<div style="font-size:70%">Recurrent Neural Networks Architecture - Credit to <a href='https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0180944'>Bao et al.</a></div>
At each step a set of values are stored as state of the model ($s$). The state is updated at each step and based on the value of the state the output of the model is calculated by passing state through the output layer ($V$). Also, the model takes an input $x_t$ and passes it through an input layer ($U$), The output of this layer is combined with the state of previous step and creates the new state ($s_t$). What is important here is that the input and output layers are the same for all the sequences of data. What makes the output different at each step are the input values and state of the model. <br>
Various architectures handel updating the state value differently and this is the main difference between Simple RNN, LSTM, and GRU. 

## Single Variate Time Series
 Recurrent Neural Networks can be quite useful for time series forecasting due to their ability to understand sequential nature of data. For the start we can see how an LSTM can forcast a time series with only one variable.

### Predicting Sine Function
In the first example we will use an LSTM model to predict the value of sine function. Since the sine function is easy to calculate we can easily evaluate the performance of the model. 

In [ ]:
import torch
from torch import nn, optim
from torch import functional as F
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

Let's create some data for training:

In [ ]:
t = np.linspace(0, 100, 1000)
sine = np.sin(t)

In [ ]:
plt.plot(t, sine)

#### Creating the model
In an LSTM the model takes one data point at the time and produces an output. This output is also called the hidden state. We can use the hidden state to calculate the next value in the sequence. The hidden state helps the model to remember what happened before and not just look at each point as an isolated instance. <br>
Pytorch LSTM component can needs a few inputs:
- __input_size:__ The size of the input data. In this example since we are dealing with a single variate data we pass in one value at a time, therefore, the input size is one.
- __hidden_size:__ This represents how many values should be stored as the hidden state in the model. Larger values allows model to understand more complex patterns in the data.
- __num_layers:__ This value refers to the number of layers in the LSTM block. Similar to what we saw in tabular data, having multiple layers allows the model to work with more complex patterns.

When we are creating our model we add a linear layer as well which will be our output layer. The output layer converts the hidden state to output value. Depending on what we want the output to be we can choose the size of output as well. In this example, we are trying to find the next value in the series, therefore, the model only returns a single value.

TODO, Mike add fig
TODO Make a coherent story
- there are multiple ways to frame a timeseries prediction problem
    - windowed - fixed content length
    - recursive - promises to let the network decide on context length
- couldn't get recursive timeseries prediction to work with just a linear layer. The problem is exploding and vanishing gradients.
    - LSTM gives it ability to forget and remember
- LSTM's have got SOTA result on many datasets. they are quite robust, and win a lot of competitions.

In [ ]:
# TODO use just seq(LSTM, linear)

class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size=1):
        super(LSTM, self).__init__()
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
        )
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        device = next(iter(self.parameters())).device
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device)
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).to(device)
        output, (h_n, c_n) = self.lstm(x, (h_0, c_0))
        h_n = h_n.view(-1, self.num_layers, self.hidden_size)[:, -1]
        out = self.linear(h_n)
        return out

In [ ]:
# lstm = nn.LSTM(
#             input_size=3,
#             hidden_size=44,
#             num_layers=2,
#             batch_first=True,
# )
# lstm?

### Preparing Data
The data for an LSTM model is a bit different from what we have seen so far. Since this model is supposed to look at a few data points to predict the next one, data needs to be broken down to smaller sections. At each step we pass a few data points into the model and the model will forecast the next. The longer this sequence of input is the better the model will be, however, this also means we require larger memory and longer training time.<br>
The function below will create the inputs and targets for the model.


In [ ]:
def create_seq_data(series, seq_length):
    sequence = []
    target = []
    for i in range(len(series) - 1 - seq_length):
        sequence.append(series[i : i + seq_length])
        target.append(series[i + 1])
    return np.array(sequence), np.array(target)

Let's split the data into a training and test set. Note that here `x` contains sets of 5 consecutive numbers and `y` contains the respective number that comes after each serie.

In [ ]:
seq_length = 5
x, y = create_seq_data(sine, seq_length)

xtrain = x[:600, :]
ytrain = y[:600]

xtest = x[600:, :]
ytest = y[600:]

Let's print xtrain to better understand what it contains.

In [ ]:
print(xtrain)

Now we can create an instance of the model with input size of one (single variate series), one layer LSTM with hidden state size of 50.

In [ ]:
model = LSTM(input_size=1, hidden_size=50, num_layers=1)

Also we need to choose the optimiser, and loss function. Let's use mean square error loss and ADAM optimiser.

In [ ]:
loss_func = torch.nn.MSELoss(reduction="mean")
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

#### Training the model
Now that all the components are ready we can train the model.

In [ ]:
epochs = 500
model.train()
for epoch in tqdm(range(epochs)):
    input_values = torch.Tensor(xtrain).unsqueeze(-1)
    optimizer.zero_grad()
    preds = model(input_values)
    loss = loss_func(preds, torch.Tensor(ytrain).unsqueeze(-1))
    loss.backward()
    optimizer.step()
    if epoch % 50 == 0:
        print('#{} Loss = {:.5f}'.format(epoch, loss.item()))

Note that when using `xtrain` and `ytrain` we first converted them to tensors, then used `unsqueeze(-1)`. `unsqueeze(-1)` adds a dimension to the end of the tensor. Why did we do that? The reason is to specify that the model is dealing with only one value.

In [ ]:
print("Normal shape:")
print(torch.Tensor(xtrain).shape)
print("\nUnsqueezed(-1) shape")
print(torch.Tensor(xtrain).unsqueeze(-1).shape)

Prior to unsqueezing the shape is the number of datapoints by the length of sequences. But after unsqueezing we have `1` as the last dimension which specifies that we are passing in one value. When we unsqueeze `y` we are specifying that the output is one value.

#### Forecasting
Now let's look at the results and see how good the model is at forecasting.

In [ ]:
model_input = torch.Tensor(xtest).unsqueeze(-1)
pred_test = model(model_input)

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(to_numpy(pred_test), "x", label="Predictions")
plt.plot(ytest, label="Target")
plt.legend()

### Predicting Customers
Let's start working with a real time series data with seasonality and trend. Unlike traditional techniques we don't need to tell the model about the trend and seasonality. LSTM will be able to recognise the patterns on its own.

#### Preparing Data

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("../../data/processed/Customers.csv")

In [ ]:
df["Customers"].plot()

To prepare the data for the model it is highly recommended to normalise the data. There are different ways to do that. You can use standard scaling or min-max scaling, etc. The advantage of normalisation is that the model doesn't need to deal with extremely large or extremely small values, which makes it more likely the model is able to forecast based on the pattern. Here, we will use `MinMaxScaler` from Scikit Learn package.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
data = scaler.fit_transform(df[["Customers"]])

Now we need to decide how many values to use for predicting. Here we are going to use 12, since we know data is monthly and has annual seasonality. Therefore seeing the past 12 months allows the model the see a full perioid. We can also choose larger or smaller values and see how it would affect the results.

In [ ]:
seq_length = 12
x, y = create_seq_data(data, seq_length)
x.shape, y.shape

Note that the shape of `x` and `y` are now suitable for the model and we won't need to unsqueeze them anymore.

In [ ]:
xtrain = x[:100, :, :]
ytrain = y[:100, :]
xtest = x[100:, :, :]
ytest = y[100:, :]

#### Training model
We are going to use the same class as the last example so we only need to create a new instance of the same model.

In [ ]:
model = LSTM(1, 50, 1)
loss_func = torch.nn.MSELoss(reduction="mean")  # mean-squared error for regression
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
def train_model(model, x, y, epochs=1):
    model.train()
    for epoch in tqdm(range(epochs)):
        input_values = torch.Tensor(x)
        optimizer.zero_grad()
        preds = model(input_values)
        loss = loss_func(preds, torch.Tensor(y))
        loss.backward()
        optimizer.step()
        if (epoch + 1) % 50 == 0:
            print('#{} Loss = {:.5f}'.format(epoch + 1, loss.item()))

In [ ]:
train_model(model, xtrain, ytrain, 500)

#### Results
Let's check out how model performs when forecasting.

In [ ]:
def test_model(model, x, y):
    model.eval()
    model_input = torch.Tensor(x)
    pred_test = model(model_input)
    plt.figure(figsize=(15, 5))
    plt.plot(tp_numpy(pred_test), ":x", label="Predictions")
    plt.plot(y, label="Target")
    plt.legend()

In [ ]:
test_model(model, xtrain, ytrain)

In [ ]:
test_model(model, xtest, ytest)

The predictions are not too bad but can certainly be improved. In most cases the model seems to underestimate the values. There are a number of techniques we can use to improve the results. One would be removing the trend from the data. By removing the trend the remaining values will be oscillating around zero and the pattern is much easier for the model to predict.<br>
Another way to improve the model is by changing the size of the model. This can be very effective as it allows the model to handle more complex patterns.

#### Exercise 1
As an exercise try to improve the performance of the model:
1. Increase the sequence length from 12 to 18
2. Increase the hidden size of the model from 50 to 100

In [ ]:
# Code Here

## Multivariate Time Series
Now that we know how to train a model using LSTM let's start working on more difficult problems. In multivariate time series, as the name suggests, we have to deal with multiple values at a time. We need to find out how these values affect each other and forecast the next step. There are many variations to this problem. In this example we will try to forecast the value of a set of variables in the next time step. The goal is to forecast Temperature, Pressure, Relative Humidity, Wind Speed, and Dew Point, using their previous and current values.

#### Preparing Data

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("../../data/processed/Generated/energy_weather.csv")

In [ ]:
df.head()

Select the columns we need.

In [ ]:
data = df[["T_out", "P_mbar", "RelH_out", "Windspeed", "Tdew"]]

This time we will use standard scaling to normalise the data.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

normalised = scaler.fit_transform(data)

In [ ]:
plt.plot(normalised, alpha=0.5)

Similar to single variate time series, we need to create data in a form that each instance of data has the values for the past few time steps.

In [ ]:
def create_multi_seq_data(series, seq_length):
    sequence = []
    target = []
    for i in range(len(series) - 1 - seq_length):
        sequence.append(series[i : i + seq_length, :])
        target.append(series[i + 1, :])
    return np.array(sequence), np.array(target)

Let's use 2 hours history for prediction. Therefore, the sequence length would be 12 (data points are recorded every 10 minutes).

In [ ]:
seq_length = 12
x, y = create_multi_seq_data(normalised, seq_length)

# We take 90% of data for training and the rest for testing
cutoff = int(len(x) * 0.9)

xtrain = x[:cutoff, :, :]
ytrain = y[:cutoff, :]

xtest = x[cutoff:, :, :]
ytest = y[cutoff:, :]

#### Training model
We can still use the same class of model. We only need to set the size of input and output.

In [ ]:
model = LSTM(5, 100, 1, 5)
loss_func = torch.nn.MSELoss(reduction="mean")  # mean-squared error for regression
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
def train_model(model, x, y, epochs=1):
    model.train()
    for epoch in tqdm(range(epochs)):
        input_values = torch.Tensor(x)
        optimizer.zero_grad()
        preds = model(input_values)
        loss = loss_func(preds, torch.Tensor(y))
        loss.backward()
        optimizer.step()
        if (epoch + 1) % 20 == 0:
            print('#{} Loss = {:.5f}'.format(epoch + 1, loss.item()), end="\r", flush=True)

In [ ]:
train_model(model, xtrain, ytrain, 100)

Note that in the examples so far we didn't print the loss value for the test set. Generally, it is recommended to uses loss value of test set to check if the model is overfitting or not.

#### Results
If the loss is low enough we can print the results and see how the model is doing in terms of forecasting.

In [ ]:
def test_model(model, x, y, plot_results=False):
    model.eval()
    model_input = torch.Tensor(x)
    pred_test = to_numpy(model(model_input))
    mse = ((pred_test - y) ** 2).mean()
    print('MSE = {:.5f}'.format(mse))
    if plot_results:
        plt.figure(figsize=(15, 5))
        plt.plot(pred_test, ":x", label="Predictions")
        plt.plot(y, label="Target")
        plt.legend()
    return pred_test

In [ ]:
preds = test_model(model, xtest, ytest, True)

In [ ]:
for i in range(5):
    plt.figure(figsize=(15, 4))
    plt.plot(ytest[:, i], label="Target")
    plt.plot(preds[:, i], label="Prediction")
    plt.legend()

## Classification
So far we tried single and multivariate time series using LSTM. But LSTMs are very flexible and useful for wide range of problems. For instance, we can have multi input and single output, or use a series to predict an entire different series. We can also use LSTM for classification of time series. What makes LSTM (or other types of Recurrecnt Neural Networks) interesting is that they are not just useful for time series. They can be used for any data that has sequence of values. For instance, they can be used for text prediction. If we map every word to a number then you can turn a text into a series of numbers then LSTM can be used for predicting the next word.


In this example we are going to look at well logs which are sequential data as well.

In [ ]:
import pandas as pd

df = pd.read_parquet(
    "../../data/processed/geolink_norge_dataset/geolink_norge_well_logs_train.parquet"
).set_index(["Well", "DEPT"])
df['Well'] = df.index.get_level_values(0)
df['DEPT'] = df.index.get_level_values(1)
df.head()

In [ ]:
# We  will stick to a gorup of long wells
df=df[df['Well'].str.startswith('35')]

In [ ]:
df['LITHOLOGY_GEOLINK']

In [ ]:
# Remove unused categories
df['LITHOLOGY_GEOLINK'] = df['LITHOLOGY_GEOLINK'].values.remove_unused_categories()

# sort categories (leads to nicer histograms)
i = df['LITHOLOGY_GEOLINK'].values
litho_sorted = i.value_counts().sort_values(ascending=True).index
df['LITHOLOGY_GEOLINK'] = i.reorder_categories(litho_sorted, ordered=True)

df['LITHOLOGY_GEOLINK'] 

In [ ]:
# Add a well int, so the model will know what well we use
well_index = df.index.get_level_values(0)
well_int = well_index.rename_categories(range(len(well_index.categories))).astype(int)
df['Well_int']= well_int
df[['Well_int']]

In [ ]:
# Get a list of wells, ordered by frequency
well_counts = df['Well'].value_counts()
wells = list(well_counts.index)
well_counts

In [ ]:
# Select the N longest well logs
n_wells = 10
selected_wells = wells[:n_wells]
df = df.loc[selected_wells]
df

In [ ]:
df['LITH_ABV'] = df["LITHOLOGY_GEOLINK"].shift().fillna('Shaly Silt')
df['LITH_ABV_INT'] = df['LITH_ABV'].values.codes
df[['LITHOLOGY_GEOLINK', 'LITH_ABV']]

In [ ]:
# # We will train on measurements above Xkm depth, and test on deeper ones
# d = df['DEPT']
# test_depth = d.mean() + d.std()/2
# print(test_depth)
# df_train = df[df['DEPT']<test_depth]
# df_test = df[df['DEPT']>test_depth]
# df_train.shape, df_test.shape

In [ ]:
# We will train on measurements above Xkm depth, and test on deeper ones
# d = df['DEPT']
# m = d.groupby(level=0).mean().dropna()
# s = d.groupby(level=0).std().dropna()
# test_depth = m+s/2
# test_depth

def get_test(x):
    d = x['DEPT']
    thresh = np.round(d.mean() + d.std() / 2)
    x['thresh'] = thresh
    return x[d>thresh]

def get_train(x):
    d = x['DEPT']
    thresh = np.round(d.mean() + d.std() / 2)
    x['thresh'] = thresh
    return x[d<=thresh]

df_test = df.groupby(level=0).apply(get_test)
df_train = df.groupby(level=0).apply(get_train)
print('train', df_train.shape, 'test', df_test.shape)
print('Train {:.0%}, test {:.0%}'.format(len(df_train) / len(df), len(df_test) / len(df)))

We will be using depth and other measurements to determine the lithology. We dealt with the same problem in the tablular data. But in tabular data we only look at the measurements at each depth to find the class, while here we can look at the variations in the measurements as well.

And add depth as a feature column:

As usual we need to create a training and test set. here we will only use `15` wells for training and `15` for testing as using the entire dataset means we need to spend much longer time for training.

We need to process the input and target data. The input data needs to be normalised with a standard scaler, and the output data needs to be converted from text to numbers. To convert text to numbers we use `LabelEncoder` from Scikit Learn.

In [ ]:
# # df['LITH_ID'] = encoder.transform(df.loc[:, "LITHOLOGY_GEOLINK"])
# # df['LITH_ID'] = encoder.transform(df.loc[:, "LITHOLOGY_GEOLINK"])
# df['LITH_ABV'] = df["LITHOLOGY_GEOLINK"].shift().fillna('Shaly Silt')
# df['LITH_ABV_INT'] = df['LITH_ABV'].values.codes
# df[['LITHOLOGY_GEOLINK', 'LITH_ABV']]

In [ ]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

scaler = StandardScaler()

# Make a encoder, that order by frequency
encoder = LabelEncoder()

# TODO need to embed prev val..., one hot, then append
encoder.classes_ = df["LITHOLOGY_GEOLINK"].values.categories # np.array(counts.index.values)
print(encoder.classes_)
feat_cols = ["CALI", "RHOB", "GR", "DTC", "RDEP", "RMED", "DEPT", "Well_int", "LITH_ABV_INT"]
scaler.fit(df[feat_cols].values)

`LabelEncoder` converts each type to a value.

In [ ]:
encoder.transform(["Shaly Silt"])

Now we can check the types at various depths:

In [ ]:
# TODO nice plot, vertical, facies colors
plt.figure(figsize=(16, 5))
idx = 0
x = df.loc[wells[idx], "DEPT"]
y = encoder.transform(df.loc[wells[idx], "LITHOLOGY_GEOLINK"])
plt.plot(x, y)

The output of a classification model is a value for each type. The type with the highest value is the one the model thinks is most likely to be associated with the input data. Therefore, the output size of the model should be the number of types.

In [ ]:
output_size = len(df["LITHOLOGY_GEOLINK"].unique())

Let's create training and test set, similar to what we had in multivariate time series. The only difference is here for each sequence of values we want the model to predict a value for each type.

In [ ]:
seq_length = 100

x_train = []
y_train = []
features = scaler.transform(df_train.loc[:, feat_cols].values)
targets = encoder.transform(df_train.loc[:, "LITHOLOGY_GEOLINK"])
for i in range(len(targets) - seq_length):
    xi = features[i : i + seq_length, :]
    yi = targets[i + seq_length - 1]
    x_train.append(xi)
    y_train.append(yi)

x_test = []
y_test = []
features = scaler.transform(df_test.loc[:, feat_cols].values)
targets = encoder.transform(df_test.loc[:, "LITHOLOGY_GEOLINK"])
for i in range(len(targets) - seq_length):
    xi = features[i : i + seq_length, :]
    yi = targets[i + seq_length - 1]
    x_test.append(xi)
    y_test.append(yi)

In [ ]:
# x_test.mean(), x_test.std()

It is important that we make sure the training and test set have close distribution. For instance, if there is a certain type in test data that doesn't exist in training data, the model will not be able to predict it.

In [ ]:
def to_numpy(x):
    if isinstance(x, torch.Tensor):
        x = x.cpu().detach().numpy()
    return x

def show_distribution(y):
    y = to_numpy(y)
    plt.hist(y, output_size * 2)
    plt.xticks(ticks=range(len(encoder.classes_)), labels=encoder.classes_, rotation=90)

In [ ]:
show_distribution(y_train)

In [ ]:
show_distribution(y_test)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
x_train = torch.Tensor(x_train).to(device)
y_train = torch.LongTensor(y_train).to(device)
x_test = torch.Tensor(x_test).to(device)
y_test = torch.LongTensor(y_test).to(device)
x_train.shape, y_train.shape, x_test.shape, device

We can still use the same class for the model. Here the input size is the number of features we are using to predict the type, and output size is the number of types. 

In [ ]:
model = LSTM(
    input_size=len(feat_cols),
    hidden_size=16,
    num_layers=3,
    output_size=output_size,
)
model = model.to(device)
model

In [ ]:
unique, counts = np.unique(to_numpy(y_train), return_counts=True)
weight = torch.from_numpy(1/(counts+1e3))
weight = None

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
torch.nn.CrossEntropyLoss(weight=weight)
loss_func = torch.nn.CrossEntropyLoss()

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
def training_loop(epochs=1, bs=128):
    pbar1 = tqdm(total=epochs)
    pbar2 = tqdm(total=len(x_train) // bs)
    all_losses = []
    all_f1s = []
    try:
        for epoch in range(epochs):
            model.train()
            training_loss = []
            training_f1 = []
            pbar2.reset(len(x_train) // bs)
            for i in range(0, len(x_train), bs):
                optimizer.zero_grad()
                preds = model(x_train[i : i + bs, ...])
                loss = loss_func(preds, y_train[i : i + bs])
                loss.backward()
                optimizer.step()
                training_loss.append(loss.item())
                f1 = f1_score(
                    to_numpy(y_train[i : i + bs]), to_numpy(preds).argmax(-1), average='weighted'
                )
                training_f1.append(f1)
                pbar2.update(1)
#                 print(
#                     f"Epoch {epoch+1}/{epochs}: Loss = {loss.item():.2f}, f1 = {f1:.3f}",
#                     end="\r",
#                     flush=True,
#                 )

            print(
                'Epoch {}/{}: Training Loss = {:.2f}, f1 = {:.3f}'.format(epoch + 1, epochs, np.mean(training_loss), np.mean(f1))
            )

            model.eval()
            test_loss = []
            test_f1 = []
            pbar2.reset(len(x_test) // bs)
            for i in range(0, len(x_test), bs):
                preds = model(x_test[i : i + bs, ...])
                loss = loss_func(preds, y_test[i : i + bs])
                test_loss.append(loss.item())
                f1 = f1_score(
                    to_numpy(y_test[i : i + bs]), to_numpy(preds.argmax(-1)), average='weighted'
                )
                test_f1.append(f1)
                pbar2.update(1)
            print(
                'Epoch {}/{}: Test Loss = {:.2f}, f1 = {:.3f}'.format(epoch + 1, epochs, np.mean(test_loss), np.mean(test_f1))
            )
            print("-" * 50)
            all_losses.append([np.mean(training_loss), np.mean(test_loss)])
            all_f1s.append([np.mean(training_f1), np.mean(test_f1)])
            pbar1.update(1)
    except KeyboardInterrupt:
        pass

    # Visualising the results
    all_losses = np.array(all_losses)
    plt.plot(all_losses[:, 0], label="Training")
    plt.plot(all_losses[:, 1], label="Test")
    plt.title("Loss")
    plt.legend()
    plt.figure()
    all_f1s = np.array(all_f1s)
    plt.plot(all_f1s[:, 0], label="Training")
    plt.plot(all_f1s[:, 1], label="Test")
    plt.title("f1")
    plt.legend()

Let's train for 10 epochs

In [ ]:
training_loop(30, 1024)

In [ ]:
preds = to_numpy(model(x_test).argmax(axis=-1))
true = to_numpy(y_test)

In [ ]:
# Baseline
f1_score(true[1:],
    true[:-1:], average='weighted')

In [ ]:
# ours
f1_score(true,
    preds, average='weighted')

In [ ]:
plt.hist(preds, bins=output_size * 2)
plt.title("Model Predictions")
plt.xticks(np.arange(output_size))
plt.figure()
plt.hist(true, bins=output_size * 2)
plt.title("Targets")
plt.xticks(np.arange(output_size))
1

In [ ]:
import pandas as pd
import sklearn.metrics
import numpy as np

def classification_report(*args, **kwargs):
    out_df = pd.DataFrame(sklearn.metrics.classification_report(*args, **kwargs, output_dict=True)).T
    # Order cols
    out_df[["precision","recall","f1-score","support"]]  
    # Round
    out_df[["precision","recall","f1-score"]]= out_df[["precision","recall","f1-score"]].apply(lambda x: round(x,2))
    out_df[["support"]]= out_df[["support"]].apply(lambda x: x.astype(np.int))
    return out_df

In [ ]:
df_report = classification_report(preds, true, labels=range(len(encoder.classes_)), target_names=encoder.classes_)
df_report

In [ ]:
cm = sklearn.metrics.confusion_matrix(preds, true, labels=range(len(encoder.classes_)))

plt.figure(figsize=(20, 20))
plt.title('Confusion Matrix')
ax=plt.gca()
disp = sklearn.metrics.ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=encoder.classes_)
disp.plot(ax=ax, xticks_rotation=90)
plt.show()

As we saw in the previous examples the model requires training over 100s of epochs to reach the best results. However, in this example due to large size of dataset and the model we stopped after `10` epochs. Try increasing the number of epochs to see how it will affect the accuracy.<br>

Obviously the model right now is not performing well. But there are number ways we can improve it:
1. Training for longer. instead of stopping after `10` epochs go for longer.
2. Increase the hidden state size.
3. Increase the size of training data by adding data from more wells to training. 
4. Increase the size of the sequences so the model get to look further in the history.

#### Exercise 2
Try one of the options above to improve the model.

In [ ]:
# Code Here

Let's have a look at model's predictions.

The distribution of data shows that the model is too focused on type `17` and `18` which is why we are having low accuracy.


<div class="alert alert-success">
    
### Solutions  
<details><summary>See solutions</summary>

<details><summary>Exercise 1</summary>
<b>Increase sequence length to 18</b>

```Python
seq_length = 18
x, y = create_seq_data(data,seq_length)
xtrain = x[:100,:,:]
ytrain = y[:100,:]
xtest = x[100:,:,:]
ytest = y[100:,:]
model = LSTM(1, 50, 1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
train_model(model, xtrain, ytrain, 500)
```
<b>Change model size to 100</b>
```Python
model = LSTM(1, 100, 1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
train_model(model, xtrain, ytrain, 500)
```

</details>
<details><summary>Exercise 2</summary>
    <b>Larger hidden size</b>
    
```Python
model = LSTM(
    input_size=len(feat_cols),
    hidden_size=400,
    num_layers=1,
    output_size=output_size,
).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
training_loop(10, 1024)
```

<b>Train longer</b>
    
```Python
model = LSTM(
    input_size=len(feat_cols),
    hidden_size=200,
    num_layers=1,
    output_size=output_size,
).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
training_loop(20, 1024)
```
 
</details>
</details>
</div>

## Further Reading
- [Introduction to RNN](http://slazebni.cs.illinois.edu/spring17/lec02_rnn.pdf)
- [A friendly introduction to Recurrent Neural Networks](https://www.youtube.com/watch?v=UNmqTiOnRfg)
- [Recurrent Neural Networks (RNN) and Long Short-Term Memory (LSTM)](https://www.youtube.com/watch?v=WCUNPb-5EYI&t=97s)
- [Introduction to LSTM](https://medium.com/x8-the-ai-community/a-7-minute-introduction-to-lstm-5e1480e6f52a)
- [LSTM and GRU](https://towardsdatascience.com/illustrated-guide-to-lstms-and-gru-s-a-step-by-step-explanation-44e9eb85bf21)
- [Time Series Prediction with LSTM](https://stackabuse.com/time-series-prediction-using-lstm-with-pytorch-in-python/)
- [Building RNN from scratch](https://medium.com/dair-ai/building-rnns-is-fun-with-pytorch-and-google-colab-3903ea9a3a79)
